<a href="https://colab.research.google.com/github/plaban1981/Langchain_usecases/blob/main/Azvaza_Evaluating_the_Ideal_Chunk_Size_for_a_RAG_System_using_LlamaIndex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install necessary packages

In [1]:
! pip  install -qU openai langchain PyPDF2 transformers tiktoken pinecone-client sentence-transformers qdrant-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 99.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 93.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.6 MB/s eta 0:00:00


In [2]:
!pip install -q protobuf==3.20.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-tools 1.59.0 requires protobuf<5.0dev,>=4.21.6, but you have protobuf 3.20.3 which is incompatible.


In [3]:
!pip install -q "unstructured[all-docs]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.1/275.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 45.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 55.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
! pip install -qU tiktoken

## Packages for scanned documents

In [5]:
!apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 18 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (7,663 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 120875 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [6]:
! apt-get install poppler-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 18 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.2 [186 kB]
Fetched 186 kB in 0s (905 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 120922 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.2_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.2) ...
Setting up poppler-utils (22.02.0-2ubuntu0.2) ...
Processing triggers for man-db (2.10.2-1) ...


In [7]:
! pip install -qU pytesseract

In [8]:
! pip3 install -qU python-docx

## Import Necessary Packages

In [13]:
from qdrant_client import models, QdrantClient
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.vectorstores.qdrant import Qdrant
import PyPDF2
from PyPDF2 import PdfReader
from tqdm.auto import tqdm
from uuid import uuid4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import MarkdownTextSplitter
from langchain.text_splitter import LatexTextSplitter
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd
from time import time,sleep
import openai
from langchain import VectorDBQA
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
import tiktoken
#
import os
import json
#
import os
from docx import Document

## Connect to drive

In [12]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## Process Documents in .PDF Format

In [27]:
def read_content(pdf_path,pdf_files):
  content = []
  for files in pdf_files:
    print(files)
    if files.split(".")[-1] == 'pdf':
      file_path = os.path.join(pdf_path,files)
      loader = UnstructuredPDFLoader(file_path)
      docs = loader.load()
      content.append(docs[0])
  return content
#
def get_text_chunks(text):
    #text_splitter = RecursiveCharacterTextSplitter(
    #text_splitter = MarkdownTextSplitter(
    text_splitter = LatexTextSplitter(
                            chunk_size=1000,
                            chunk_overlap=0)
    chunks = text_splitter.split_text(text)
    return chunks
#

In [77]:
import re
def remove_whitespace_special_characters(text):
    # Remove whitespace and special characters using regex
    cleaned_text = re.sub(r'\s+', ' ', text)  # Remove multiple whitespaces and replace with a single space
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', cleaned_text)  # Remove special characters
    cleaned_text =  re.sub(r'oO', '\n', cleaned_text)
    cleaned_text =  re.sub(r'©', '\n', cleaned_text)
    return cleaned_text

In [36]:
def preprocess_function(request_data,content_type=None):
  pdf_docs = request_data["pdf1"]
  prompt = request_data["query"]
  topic = request_data["topic"]
  game = request_data["game"]
  ncert = request_data["pdf2"]
  interest = request_data["USER_INTEREST"]
  #
  if pdf_docs != " ":
    pdf_files = [f for f in os.listdir(pdf_docs)]
    #
    chapter_content = read_content(pdf_docs,pdf_files)

  if ncert != " ":
    ncert_files = [f for f in os.listdir(ncert)]
    #
    ncert_content = read_content(ncert,ncert_files)


  #
  metadatas = []
  texts = []
  p = 0
  #processing Azvaza content
  if pdf_docs  != " ":
    for record in tqdm(chapter_content):
        p +=1
        # first get metadata fields for this record
        metadata = {'page number': p}
        # now we create chunks from the record text
        record_page_content = remove_whitespace_special_characters(record.page_content)
        record_texts = get_text_chunks(record.page_content)
        #print(f"Record Text : \n{record_texts}")
        # create individual metadata dicts for each chunk
        record_metadatas = [{
            "chunk": j,"source":"Azvasa","text": text, **metadata
        } for j, text in enumerate(record_texts)]
        # append these to current batches
        texts.extend(record_texts)
        metadatas.extend(record_metadatas)
  #processing ncert content
  if ncert != " ":
    for record in tqdm(ncert_content):
        p +=1
        # first get metadata fields for this record
        metadata = {'page number': p}
        # now we create chunks from the record text
        record_texts = get_text_chunks(record.page_content)
        # create individual metadata dicts for each chunk
        record_metadatas = [{
            "chunk": j,"source":"ncert","text": text, **metadata
        } for j, text in enumerate(record_texts)]
        # append these to current batches
        texts.extend(record_texts)
        metadatas.extend(record_metadatas)
  #
  c = []
  for i in range(len(metadatas)):
    c.append(metadatas[i]['text'])

  page_content = c
  return (chapter_content,page_content,metadatas,prompt,topic,game,interest)

In [70]:
request_data = {"pdf1":"/content/drive/MyDrive/Plants",
                "pdf2":" ",
                "query":"Please generate a detailed study material for the student for the topic 'Plants' which is factually correct and based on the information retrieved from the vector store. Also provide detailed examples for the concepts explained in a step by step manner.",
                "topic": "Fun With Magnets",
                "USER_INTEREST":"Emma Watson",
                "game":"Harry Potter Stories"}
#
content = preprocess_function(request_data,content_type=None)


plants_page1.pdf
plant_page_2.pdf
plants_page3.pdf
plants_page4.pdf
plants_page5.pdf
plants_page6.pdf
plants_page7.pdf
plants_page8.pdf
plants_page9.pdf
plants_page10.pdf
plants_page11.pdf
plants_page12.pdf
plants_page13.pdf
plants_page14.pdf
plants_page15.pdf
plants_page16.pdf
plants_page17.pdf


  0%|          | 0/17 [00:00<?, ?it/s]

In [71]:
chapter_content,page_content,metadatas,prompt,topic,game,interest = content

## MarkdownTextSplitter

In [24]:
for pages in page_content:
  print(remove_whitespace_special_characters(pages))


 Types of Plants  Root  Stem  Leaves  Flowers fruits and seeds CHAPTER 2 PLANTS Overview There are many kinds of plants Some have flowers some dont some give us fruits some give us vegetables some have thorns and some are poisonous too But they are also alike in many ways Plants have roots Plants have stems Plants have leaves Plants need soil sun and water to live Plants make their own food photosynthesis Plants give us oxygen We will study what is common to all plants in this lesson Types of Plants If you look around carefully all the plants have so many variations Each plant has different shape size habitat and so on Plants grow on land as well as in water Based on the structure of a lants they can be classified into these following categories Trees
Trees are big tall and strong plants They have a strong thick and brown stem called the the trunk They have many branches that can hold leaves flowers and fruits They also have very thick and strong roots Examples Neem and Banyan  Humili

## Latexsplitter

In [78]:
for pages in page_content:
  print(remove_whitespace_special_characters(pages))


 Types of Plants  Root  Stem  Leaves  Flowers fruits and seeds CHAPTER 2 PLANTS Overview There are many kinds of plants Some have flowers some dont some give us fruits some give us vegetables some have thorns and some are poisonous too But they are also alike in many ways Plants have roots Plants have stems Plants have leaves Plants need soil sun and water to live Plants make their own food photosynthesis Plants give us oxygen We will study what is common to all plants in this lesson Types of Plants If you look around carefully all the plants have so many variations Each plant has different shape size habitat and so on Plants grow on land as well as in water Based on the structure of a lants they can be classified into these following categories Trees Trees are big tall and strong plants They have a strong thick and brown stem called the the trunk They have many branches that can hold leaves flowers and fruits They also
have very thick and strong roots Examples Neem and Banyan  Humili

In [33]:
page_content[0]

'oO\n\nTypes of\n\nPlants\n\n©\n\nRoot\n\n©\n\nStem\n\n©\n\nLeaves\n\n©\n\nFlowers,\n\nfruits and\n\nseeds\n\nCHAPTER 2: PLANTS\n\nOverview\n\nThere are many kinds of plants. Some have flowers, some don’t, some\n\ngive us fruits, some give us vegetables, some have thorns and some are\n\npoisonous too! But they are also alike in many ways.\n\nPlants have roots\n\nPlants have stems\n\nPlants have leaves\n\nPlants need soil, sun and water to live\n\nPlants make their own food (photosynthesis)\n\nPlants give us oxygen\n\nWe will study what is common to all plants in this lesson.\n\nTypes of Plants\n\nIf you look around carefully, all the plants have so many variations.\n\nEach plant has different shape, size, habitat and so on. Plants grow on\n\nland as well as in water. Based on the structure of a lants they can be\n\nclassified into these following categories:\n\nTrees\n\nTrees are big, tall and strong plants. They have a strong, thick and brown stem called the the trunk. They have many 

In [34]:
metadatas[0]

{'chunk': 0,
 'source': 'Azvasa',
 'text': 'oO\n\nTypes of\n\nPlants\n\n©\n\nRoot\n\n©\n\nStem\n\n©\n\nLeaves\n\n©\n\nFlowers,\n\nfruits and\n\nseeds\n\nCHAPTER 2: PLANTS\n\nOverview\n\nThere are many kinds of plants. Some have flowers, some don’t, some\n\ngive us fruits, some give us vegetables, some have thorns and some are\n\npoisonous too! But they are also alike in many ways.\n\nPlants have roots\n\nPlants have stems\n\nPlants have leaves\n\nPlants need soil, sun and water to live\n\nPlants make their own food (photosynthesis)\n\nPlants give us oxygen\n\nWe will study what is common to all plants in this lesson.\n\nTypes of Plants\n\nIf you look around carefully, all the plants have so many variations.\n\nEach plant has different shape, size, habitat and so on. Plants grow on\n\nland as well as in water. Based on the structure of a lants they can be\n\nclassified into these following categories:\n\nTrees\n\nTrees are big, tall and strong plants. They have a strong, thick and brown

In [41]:
doc = chapter_content[0]

In [45]:
doc

Document(page_content='oO\n\nTypes of\n\nPlants\n\n©\n\nRoot\n\n©\n\nStem\n\n©\n\nLeaves\n\n©\n\nFlowers,\n\nfruits and\n\nseeds\n\nCHAPTER 2: PLANTS\n\nOverview\n\nThere are many kinds of plants. Some have flowers, some don’t, some\n\ngive us fruits, some give us vegetables, some have thorns and some are\n\npoisonous too! But they are also alike in many ways.\n\nPlants have roots\n\nPlants have stems\n\nPlants have leaves\n\nPlants need soil, sun and water to live\n\nPlants make their own food (photosynthesis)\n\nPlants give us oxygen\n\nWe will study what is common to all plants in this lesson.\n\nTypes of Plants\n\nIf you look around carefully, all the plants have so many variations.\n\nEach plant has different shape, size, habitat and so on. Plants grow on\n\nland as well as in water. Based on the structure of a lants they can be\n\nclassified into these following categories:\n\nTrees\n\nTrees are big, tall and strong plants. They have a strong, thick and brown stem called the the 

In [57]:
! pip install -qU pymupdf



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 32.5 MB/s eta 0:00:00


In [61]:
import fitz  # PyMuPDF

def remove_headers_footers(input_pdf_path, output_pdf_path):
    # Open the PDF
    pdf_document = fitz.open(input_pdf_path)

    for page_number in range(pdf_document.page_count):
        page = pdf_document[page_number]

        # Set the rectangle for the header and footer area (adjust coordinates as needed)
        header_footer_rect = fitz.Rect(0, 0, page.rect.width, 100)  # Assuming the header/footer is within the first 100 points

        # Extract text in the header and footer area
        header_footer_text = page.get_text("text", clip=header_footer_rect)

        # Clear the text content in the header and footer area
        for item in header_footer_text:
            item.set_text("")

    # Save the modified PDF
    pdf_document.save(output_pdf_path)
    pdf_document.close()

# Example usage
input_pdf_path = '/content/drive/MyDrive/Plants/plant_page_2.pdf'  # Replace with the path to your input PDF
output_pdf_path = 'output.pdf'  # Replace with the desired output PDF path

remove_headers_footers(input_pdf_path, output_pdf_path)
print("Headers and footers removed. Modified PDF saved at:", output_pdf_path)


Headers and footers removed. Modified PDF saved at: output.pdf


# Evaluating the Ideal Chunk Size for a RAG System using LlamaIndex

# **Introduction**

Retrieval-augmented generation (RAG) has introduced an innovative approach that fuses the extensive retrieval capabilities of search systems with the LLM. When implementing a RAG system, one critical parameter that governs the system’s efficiency and performance is the `chunk_size`. How does one discern the optimal chunk size for seamless retrieval? This is where LlamaIndex `Response Evaluation` comes handy. In this blogpost, we'll guide you through the steps to determine the best `chunk size` using LlamaIndex’s `Response Evaluation` module. If you're unfamiliar with the `Response` Evaluation module, we recommend reviewing its [documentation](https://docs.llamaindex.ai/en/latest/core_modules/supporting_modules/evaluation/modules.html) before proceeding.

## **Why Chunk Size Matters**

Choosing the right `chunk_size` is a critical decision that can influence the efficiency and accuracy of a RAG system in several ways:

1. **Relevance and Granularity**: A small `chunk_size`, like 128, yields more granular chunks. This granularity, however, presents a risk: vital information might not be among the top retrieved chunks, especially if the `similarity_top_k` setting is as restrictive as 2. Conversely, a chunk size of 512 is likely to encompass all necessary information within the top chunks, ensuring that answers to queries are readily available. To navigate this, we employ the Faithfulness and Relevancy metrics. These measure the absence of ‘hallucinations’ and the ‘relevancy’ of responses based on the query and the retrieved contexts respectively.
2. **Response Generation Time**: As the `chunk_size` increases, so does the volume of information directed into the LLM to generate an answer. While this can ensure a more comprehensive context, it might also slow down the system. Ensuring that the added depth doesn't compromise the system's responsiveness is crucial.

In essence, determining the optimal `chunk_size` is about striking a balance: capturing all essential information without sacrificing speed. It's vital to undergo thorough testing with various sizes to find a configuration that suits the specific use-case and dataset.

## **Setup**

Before embarking on the experiment, we need to ensure all requisite modules are imported:

In [79]:
!pip install llama-index pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.7/877.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.5/276.5 kB 9.1 MB/s eta 0:00:00


In [80]:
import nest_asyncio

nest_asyncio.apply()

from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
)
from llama_index.evaluation import (
    DatasetGenerator,
    FaithfulnessEvaluator,
    RelevancyEvaluator
)
from llama_index.llms import OpenAI

import openai
import time
from getpass import getpass
openai.api_key = getpass() # set your openai api key

··········


## **Download Data**

We'll be using the Uber 10K SEC Filings for 2021 for this experiment.

In [90]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/jerryjliu/llama_index/main/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'

--2023-10-10 13:36:32--  https://raw.githubusercontent.com/jerryjliu/llama_index/main/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘data/10k/uber_2021.pdf’

data/10k/uber_2021. 100%[===================>]   1.79M  --.-KB/s    in 0.07s   

2023-10-10 13:36:33 (26.4 MB/s) - ‘data/10k/uber_2021.pdf’ saved [1880483/1880483]



## **Load Data**

Let’s load our document.

In [91]:
# Load Data

reader = SimpleDirectoryReader("./data/10k/")
documents = reader.load_data()

In [92]:
documents[0]

Document(id_='a084fc75-8fd6-4dbf-ba5b-a93b5aaa8e2b', embedding=None, metadata={'page_label': '1', 'file_name': 'uber_2021.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='8236c7b95b366f678fd506883779c1f7b8349b0d0b466a9da6694959abfab3c6', text='UNITED STATESSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\n____________________________________________ \nFORM\n 10-K____________________________________________ \n(Mark One)\n☒\n ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended\n December 31, 2021OR\n☐\n TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the transition period from_____ to _____            \nCommission File Number: 001-38902\n____________________________________________ \nUBER TECHNOLOGIES, INC.\n(Exact name of registrant as specif\nied in its charter)____________________________________________ \nDelaware\n45-2647441 (S

## **Question Generation**

To select the right `chunk_size`, we'll compute metrics like Average Response time, Faithfulness, and Relevancy for various `chunk_sizes`. The `DatasetGenerator` will help us generate questions from the documents.

In [94]:
# To evaluate for each chunk size, we will first generate a set of 40 questions from first 20 pages.
eval_documents = documents[:20]
data_generator = DatasetGenerator.from_documents(eval_documents)
eval_questions = data_generator.generate_questions_from_nodes(num = 40)

In [96]:
eval_questions

['What is the fiscal year end date for Uber Technologies, Inc.?',
 'In which state is Uber Technologies, Inc. incorporated?',
 "What is the address of Uber Technologies, Inc.'s principal executive offices?",
 "What is the trading symbol for Uber Technologies, Inc.'s common stock?",
 'Is Uber Technologies, Inc. a well-known seasoned issuer?',
 'Has Uber Technologies, Inc. filed all reports required by the Securities Exchange Act of 1934 in the past 12 months?',
 'Has Uber Technologies, Inc. been subject to filing requirements for the past 90 days?',
 'Has Uber Technologies, Inc. submitted every Interactive Data File required by Rule 405 of Regulation S-T in the past 12 months?',
 'What are the different categories of filers defined in Rule 12b-2 of the Exchange Act?',
 'Is Uber Technologies, Inc. considered a large accelerated filer, an accelerated filer, a non-accelerated filer, a smaller reporting company, or an emerging growth company?',
 'What is the filing status of the registrant 

## Setting Up Evaluators

We are setting up the GPT-4 model to serve as the backbone for evaluating the responses generated during the experiment. Two evaluators, `FaithfulnessEvaluator` and `RelevancyEvaluator`, are initialised with the `service_context` .

1. **Faithfulness Evaluator** - It is useful for measuring if the response was hallucinated and measures if the response from a query engine matches any source nodes.
2. **Relevancy Evaluator** - It is useful for measuring if the query was actually answered by the response and measures if the response + source nodes match the query.

In [97]:
# We will use GPT-4 for evaluating the responses
gpt4 = OpenAI(temperature=0, model="gpt-4")

# Define service context for GPT-4 for evaluation
service_context_gpt4 = ServiceContext.from_defaults(llm=gpt4)

# Define Faithfulness and Relevancy Evaluators which are based on GPT-4
faithfulness_gpt4 = FaithfulnessEvaluator(service_context=service_context_gpt4)
relevancy_gpt4 = RelevancyEvaluator(service_context=service_context_gpt4)


## **Response Evaluation For A Chunk Size**

We evaluate each chunk_size based on 3 metrics.

1. Average Response Time.
2. Average Faithfulness.
3. Average Relevancy.

Here's a function, `evaluate_response_time_and_accuracy`, that does just that which has:

1. VectorIndex Creation.
2. Building the Query Engine**.**
3. Metrics Calculation.

In [98]:
# Define function to calculate average response time, average faithfulness and average relevancy metrics for given chunk size
# We use GPT-3.5-Turbo to generate response and GPT-4 to evaluate it.
def evaluate_response_time_and_accuracy(chunk_size, eval_questions):
    """
    Evaluate the average response time, faithfulness, and relevancy of responses generated by GPT-3.5-turbo for a given chunk size.

    Parameters:
    chunk_size (int): The size of data chunks being processed.

    Returns:
    tuple: A tuple containing the average response time, faithfulness, and relevancy metrics.
    """

    total_response_time = 0
    total_faithfulness = 0
    total_relevancy = 0

    # create vector index
    llm = OpenAI(model="gpt-3.5-turbo")
    service_context = ServiceContext.from_defaults(llm=llm, chunk_size=chunk_size)
    vector_index = VectorStoreIndex.from_documents(
        eval_documents, service_context=service_context
    )
    # build query engine
    # By default, similarity_top_k is set to 2. To experiment with different values, pass it as an argument to as_query_engine()
    query_engine = vector_index.as_query_engine()
    num_questions = len(eval_questions)

    # Iterate over each question in eval_questions to compute metrics.
    # While BatchEvalRunner can be used for faster evaluations (see: https://docs.llamaindex.ai/en/latest/examples/evaluation/batch_eval.html),
    # we're using a loop here to specifically measure response time for different chunk sizes.
    for question in eval_questions:
        start_time = time.time()
        response_vector = query_engine.query(question)
        elapsed_time = time.time() - start_time

        faithfulness_result = faithfulness_gpt4.evaluate_response(
            response=response_vector
        ).passing

        relevancy_result = relevancy_gpt4.evaluate_response(
            query=question, response=response_vector
        ).passing

        total_response_time += elapsed_time
        total_faithfulness += faithfulness_result
        total_relevancy += relevancy_result

    average_response_time = total_response_time / num_questions
    average_faithfulness = total_faithfulness / num_questions
    average_relevancy = total_relevancy / num_questions

    return average_response_time, average_faithfulness, average_relevancy

## **Testing Across Different Chunk Sizes**

We'll evaluate a range of chunk sizes to identify which offers the most promising metrics

In [99]:
# Iterate over different chunk sizes to evaluate the metrics to help fix the chunk size.

for chunk_size in [128, 256, 512, 1024, 2048]:
  avg_response_time, avg_faithfulness, avg_relevancy = evaluate_response_time_and_accuracy(chunk_size,eval_questions)
  print(f"Chunk size {chunk_size} - Average Response time: {avg_response_time:.2f}s, Average Faithfulness: {avg_faithfulness:.2f}, Average Relevancy: {avg_relevancy:.2f}")

Chunk size 128 - Average Response time: 17.19s, Average Faithfulness: 0.88, Average Relevancy: 0.80


Chunk size 256 - Average Response time: 2.27s, Average Faithfulness: 0.88, Average Relevancy: 0.78
Chunk size 512 - Average Response time: 2.94s, Average Faithfulness: 0.88, Average Relevancy: 0.85
Chunk size 1024 - Average Response time: 2.82s, Average Faithfulness: 0.95, Average Relevancy: 0.93
Chunk size 2048 - Average Response time: 2.80s, Average Faithfulness: 0.97, Average Relevancy: 0.93


## Reading Unstructured File Reader

In [101]:
from llama_index import download_loader, SimpleDirectoryReader

UnstructuredReader = download_loader('UnstructuredReader')

In [102]:
dir_reader = SimpleDirectoryReader('/content/drive/MyDrive/Plants', file_extractor={
  ".pdf": UnstructuredReader(),
  ".html": UnstructuredReader(),
})
documents = dir_reader.load_data()

print(documents)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[Document(id_='61ce236b-611d-4c53-af7a-78aa76ef63b8', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='39eafb25f70e7d32a55ac922351d3059139ce13fd241f49d32097aaab6d325c7', text="Shrub\n\nShrubs are small or medium-sized\n\nOo\n\nplants and they are also called as bushes. Types of\n\nThey are generally smaller than trees. Plants\n\nThey have many stems that are thick\n\nand strong. Examples: Rose, Hibiscus\n\n©\n\nand Tulsi. Fig 2: Shrubs Root\n\nHerb\n\n©\n\nHerbs are very small plants. They have slender and Stem\n\ngreen stems. They are used to add flavour in food.\n\nMany herbs have medicinal value in them. The\n\n©\n\nbanana plant is the world's largest herb! Examples: L eaves\n\nMint, Coriander, and Spinach.\n\nFig 3: Mint Climbers ©) Climbers are plants with thin and Flowers, fruits and\n\ngenerally coiled stems. They cannot grow straight like a tree or shrub. They seeds need support to grow. They climb on\n\nwalls,

In [103]:
# To evaluate for each chunk size, we will first generate a set of 40 questions from first 20 pages.
eval_documents = documents[:20]
data_generator = DatasetGenerator.from_documents(eval_documents)
eval_questions = data_generator.generate_questions_from_nodes(num = 40)

In [104]:
eval_questions

['What are some examples of shrubs mentioned in the context information?',
 'How are shrubs different from trees in terms of size?',
 'What are some characteristics of herbs mentioned in the context information?',
 "Which plant is considered the world's largest herb?",
 'Name some herbs that have medicinal value.',
 'How do climbers differ from shrubs and trees in terms of growth pattern?',
 'What do climbers need in order to grow?',
 'Give examples of climbers mentioned in the context information.',
 'How do creepers differ from climbers in terms of growth pattern?',
 'Describe the growth pattern of creepers mentioned in the context information.',
 'What are the common characteristics of all plants?',
 'What are the three main parts of a plant?',
 'What are the essential requirements for plants to live?',
 'What is the process by which plants make their own food?',
 'What is the main function of leaves in a plant?',
 'How do plants contribute to the production of oxygen?',
 'How can p

## **Testing Across Different Chunk Sizes**

In [105]:
# Iterate over different chunk sizes to evaluate the metrics to help fix the chunk size.
chunk_sizes = []
response_time = []
faithfulness = []
relevancy = []

for chunk_size in [128, 256, 512, 1024, 2048]:
  avg_response_time, avg_faithfulness, avg_relevancy = evaluate_response_time_and_accuracy(chunk_size,eval_questions)
  chunk_sizes.append(chunk_size)
  response_time.append(avg_response_time)
  faithfulness.append(avg_faithfulness)
  relevancy.append(avg_relevancy)
  #
  print(f"Chunk size {chunk_size} - Average Response time: {avg_response_time:.2f}s, Average Faithfulness: {avg_faithfulness:.2f}, Average Relevancy: {avg_relevancy:.2f}")

Chunk size 128 - Average Response time: 17.50s, Average Faithfulness: 0.95, Average Relevancy: 0.95
Chunk size 256 - Average Response time: 2.34s, Average Faithfulness: 0.93, Average Relevancy: 0.93
Chunk size 512 - Average Response time: 2.61s, Average Faithfulness: 0.93, Average Relevancy: 0.95
Chunk size 1024 - Average Response time: 2.52s, Average Faithfulness: 0.95, Average Relevancy: 0.95
Chunk size 2048 - Average Response time: 2.57s, Average Faithfulness: 0.97, Average Relevancy: 0.93


## Response- Azvaza

In [106]:
result = pd.DataFrame({'Chunk_Size':chunk_sizes,'Average Response Time': response_time,'Average Faithfulness': faithfulness,'Average Relevancy': relevancy})
result


,Chunk_Size,Average Response Time,Average Faithfulness,Average Relevancy
0,128,17.501502,0.950,0.950
1,256,2.338921,0.925,0.925
2,512,2.612032,0.925,0.950
3,1024,2.521067,0.950,0.950
4,2048,2.567891,0.975,0.925


In [107]:
result.to_csv("Chunk_size_analysis.csv",index=False)